In [1]:
# !wget https://huggingface.co/datasets/mesolitica/mixtral-malaysian-general-qa/resolve/main/mixtral-conversation-badthings.jsonl

In [2]:
# !wget https://huggingface.co/datasets/mesolitica/mixtral-malaysian-general-qa/resolve/main/mixtral-conversation-stupid.jsonl
# !wget https://huggingface.co/datasets/mesolitica/mixtral-malaysian-general-qa/resolve/main/mixtral-critics-politician-multiturn.jsonl

In [3]:
import malaya
import json

/home/husein/.local/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/.local/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [8]:
text = set()
files = [
    'mixtral-conversation-stupid.jsonl',
    'mixtral-conversation-badthings.jsonl'
]

for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            for l_ in l:
                if l_['role'] != 'assistant':
                    continue

                t = l_['content_ms']
                if t is None:
                    continue

                if '-' in t:
                    continue

                if '+' in t:
                    continue

                if '*' in t:
                    continue

                if len(t) < 5:
                    continue

                if len(t) > 150:
                    continue
                text.add(t)
            
len(text)

58069

In [9]:
from tqdm import tqdm

new_text = set()
with open('mixtral-critics-politician-multiturn.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        for l_ in l:
            if l_['role'] != 'assistant':
                continue

            t = l_['content_ms']
            if t is None:
                continue

            if '+' in t:
                continue

            if '*' in t:
                continue
            
            splitted = malaya.text.function.split_into_sentences(t)
            splitted = [s for s in splitted if 10 < len(s) < 150]
            if not len(splitted):
                continue
            
            new_text.add(splitted[0])
            
len(new_text)

135770it [04:24, 513.53it/s] 


272785

In [10]:
from malaya_speech.tts import load_text_ids
normalizer = load_text_ids(pad_to = None, understand_punct = True, is_lower = False)

t, ids = normalizer.normalize('saya nak makan nasi ayam yang sedap, lagi lazat, dan hidup sangatlah susah kan.', add_fullstop = False)

`openai-whisper` is not available, native whisper processor is not available, will use huggingface processor instead.
`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.
/home/husein/.local/lib/python3.10/site-packages/malaya/normalizer/rules.py:204: FutureWarning: Possible nested set at position 42
  k.lower(): re.compile(_expressions[k]) for k, v in _expressions.items()
/home/husein/.local/lib/python3.10/site-packages/malaya/normalizer/rules.py:204: FutureWarning: Possible nested set at position 3
  k.lower(): re.compile(_expressions[k]) for k, v in _expressions.items()


In [23]:
processed_text = []
for t in tqdm(text):
    t_, ids = normalizer.normalize(t, add_fullstop = False)
    if '--' in t_:
        continue
    processed_text.append({
        'original': t,
        'normalized': t_
    })

100%|███████████████████████████████████| 58069/58069 [00:35<00:00, 1627.39it/s]


In [26]:
processed_text[-1]

{'original': 'Walaupun komputer, seperti saya, boleh memproses maklumat dan belajar daripada data, mereka tidak berfikir atau mengalami emosi seperti manusia.',
 'normalized': 'Walaupun komputer , seperti saya , boleh memproses maklumat dan belajar daripada data , mereka tidak berfikir atau mengalami emosi seperti manusia .'}

In [27]:
with open('chatbot-conversation.json', 'w') as fopen:
    json.dump(processed_text, fopen)

In [28]:
processed_text = []
for t in tqdm(new_text):
    t_, ids = normalizer.normalize(t, add_fullstop = False)
    if '--' in t_:
        continue
    processed_text.append({
        'original': t,
        'normalized': t_
    })

100%|█████████████████████████████████| 272785/272785 [02:52<00:00, 1579.39it/s]


In [29]:
processed_text[0]

{'original': 'Untuk menangani isu rasuah dan akauntabiliti dalam politik Malaysia, pelbagai pembaharuan dan langkah boleh diambil.',
 'normalized': 'Untuk menangani isu rasuah dan akauntabiliti dalam politik Malaysia , pelbagai pembaharuan dan langkah boleh diambil .'}

In [30]:
with open('chatbot-conversation-politics.json', 'w') as fopen:
    json.dump(processed_text, fopen)